# ML Pipeline Preparation

### 1. Importing libraries and loading data from data base

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime
import collections
import copy


from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.multioutput import MultiOutputClassifier
"""
from sklearn.ensemble import RandomForestClassifier
Random Forest is too time consuming for typical single machine. If you want too see the results
for the Random Forest Classifier please go back to the very first version of this project
"""
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report

### Optional display settings

In [3]:
plt.style.use('seaborn-dark')
pd.set_option("display.max_rows", 1000)

In [4]:
engine = create_engine('sqlite:///Pipeline_Project.db')
df = pd.read_sql_table('Messages', engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26028 entries, 0 to 26027
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      26028 non-null  int64  
 1   message                 26028 non-null  object 
 2   related                 26028 non-null  float64
 3   request                 26028 non-null  float64
 4   offer                   26028 non-null  float64
 5   aid_related             26028 non-null  float64
 6   medical_help            26028 non-null  float64
 7   medical_products        26028 non-null  float64
 8   search_and_rescue       26028 non-null  float64
 9   security                26028 non-null  float64
 10  military                26028 non-null  float64
 11  water                   26028 non-null  float64
 12  food                    26028 non-null  float64
 13  shelter                 26028 non-null  float64
 14  clothing                26028 non-null

### General division into labels and features

In [5]:
X = df.iloc[:, 1]
Y = df.iloc[:, 2:]

In [ ]:
### Weights adjustments

In [7]:
#class_weights = []

In [8]:
"""for feature in Y:
    dictionary = {(Y[feature].value_counts()/len(Y)).index[0]: (Y[feature].value_counts()/len(Y)).values[1],
                  (Y[feature].value_counts()/len(Y)).index[1]: (Y[feature].value_counts()/len(Y)).values[0]}

    class_weights.append(dictionary)
"""    

In [9]:
"""for dictionary in copy.copy(class_weights):
    dictionary = {k: dictionary[k] for k in sorted(dictionary)}
    
    class_weights.pop(0)
    class_weights.append(dictionary)
"""

In [10]:
"""
for element in class_weights:
    for key, value in element.items():
        print(type(key), type(value))
        
"""

'\nfor element in class_weights:\n    for key, value in element.items():\n        print(type(key), type(value))\n        \n'

In [12]:
#compute_sample_weight(class_weights, Y)

array([6.61167834e-49, 1.49963778e-46, 6.61167834e-49, ...,
       6.61167834e-49, 2.70246741e-47, 6.61167834e-49])

### 2. Key functions

#### Tokenizer for processing the text data

In [13]:
def tokenize(text):
    #Normalization - lowercase  - no punctuation removal - the nltk.tokenize should interpret them by itself
    text = text.lower()
    ######text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    #Toeknization
    ######words = text.split()
    words = word_tokenize(text)
    #Stop words removal
    words = [w for w in words if w not in stopwords.words("english")]
    
    #Named Entities??
    # Perhaps
    
    #Lemmatization and Stemming
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    stemmed  = [PorterStemmer().stem(w) for w in lemmed]
    
    return stemmed

#### Since I want to test out many models, it seems reasonable to write a function which will make pipeline with keyword arguments

In [14]:
def make_sklearn_pipeline(Pipeline = Pipeline, memory = None, verbose = False, **kwargs):
    """
    Firt argument is sklearn Pipepline class. It should not be changed.
    
    Definition of pipeline steps happens EXPLICITLY within the instantiation!
    Definition of typical pipeline looks like following:
    example_pipeline = make_sklearn_pipeline(steps = [('name#1',function#1),
                                                      ('name#2',function#2),
                                                      ('name#3',function#3),
                                                      ('name#4',function#4)....], verbose = ...,
                                                      memory = ...)
    
    It is not required to give values for ´verbose´ and ´memory´. They have default values as False
    and None respectively. For more information visit: 
    ´https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html´
    """
    pipeline = Pipeline(steps = kwargs['steps'], memory = memory, verbose = verbose)
    
    return pipeline

#### Function transforming the classification report into data frame with recall, precision and accuracy only 

In [15]:
def df_from_sklearn_cl_reports(cl_reports):

    data_frame = pd.DataFrame()

    for feature in list(cl_reports.keys()):
        l = cl_reports[feature].split(' ')
        l = [x for x in l if x !='']
        l = [x for x in l if '\n' not in x]
        l = l[:l.index('accuracy')]
        columns = l[:3]
        l = [x for x in l if x not in columns]
        labels = []
        precisions = []
        recalls = []
        f1_scores = []
    
        for i, element in enumerate(l):
            if i == 0 or i % 4 == 0:
                labels.append(element)
                precisions.append(l[i+1])
                recalls.append(l[i+2])
                f1_scores.append(l[i+3])
            
        comunicates = [feature]*len(labels)
    
        if data_frame.shape[0] == 0:      
            data_frame['communicate'] = comunicates
            data_frame['label'] = labels
            data_frame['precisions'] = precisions
            data_frame['recalls'] = recalls
            data_frame['f1_scores'] = f1_scores
        
        else:
            auxiliary_df = pd.DataFrame()
            auxiliary_df['communicate'] = comunicates
            auxiliary_df['label'] = labels
            auxiliary_df['precisions'] = precisions
            auxiliary_df['recalls'] = recalls
            auxiliary_df['f1_scores'] = f1_scores 
        
            data_frame = pd.concat([data_frame, auxiliary_df])
        
            del auxiliary_df
    
    data_frame.set_index(["communicate"], inplace = True)
    
    #Chaning the data types
    data_frame.label = data_frame.label.astype("float")
    data_frame.label = data_frame.label.astype("int")
    data_frame.precisions = data_frame.precisions.astype("float")
    data_frame.recalls = data_frame.recalls.astype("float")
    data_frame.f1_scores = data_frame.f1_scores.astype("float")
    
    return data_frame    

### 3. Building a machine learning pipeline

#### 3.1 AdaBoost Classifier

##### Definition and fitting

In [16]:
adaboost_with_transformer = make_sklearn_pipeline(verbose = True, steps = [('vect', CountVectorizer(tokenizer = tokenize)),
                           ('tfidf', TfidfTransformer()),                                              
                           ('clf', MultiOutputClassifier(estimator = AdaBoostClassifier(
                           DecisionTreeClassifier(class_weight = "balanced"))))])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3)


In [18]:
adaboost_with_transformer.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total= 3.1min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 9.6min


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002885DCEEE18>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced'))))],
         verbose=True)

##### Predicting and classification report

In [19]:
y_pred = adaboost_with_transformer.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))

related
              precision    recall  f1-score   support

         0.0       0.55      0.44      0.49      1895
         1.0       0.83      0.88      0.86      5914

    accuracy                           0.78      7809
   macro avg       0.69      0.66      0.67      7809
weighted avg       0.76      0.78      0.77      7809

request
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91      6478
         1.0       0.55      0.64      0.59      1331

    accuracy                           0.85      7809
   macro avg       0.74      0.77      0.75      7809
weighted avg       0.86      0.85      0.85      7809

offer
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      7771
         1.0       0.01      0.03      0.01        38

    accuracy                           0.98      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.99      0.98      0.98      780

              precision    recall  f1-score   support

         0.0       0.96      0.93      0.95      7379
         1.0       0.22      0.32      0.26       430

    accuracy                           0.90      7809
   macro avg       0.59      0.63      0.60      7809
weighted avg       0.92      0.90      0.91      7809

direct_report
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87      6256
         1.0       0.48      0.52      0.50      1553

    accuracy                           0.79      7809
   macro avg       0.68      0.69      0.69      7809
weighted avg       0.80      0.79      0.80      7809



##### Providing a data frame for easier results assessment

In [20]:
classification_reports = {}

In [21]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]));

In [22]:
adaboost_with_transformer_report_df = df_from_sklearn_cl_reports(classification_reports)

In [39]:
adaboost_with_transformer_report_df.loc[
    adaboost_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False).mean()

0.3465714285714285

In [40]:
adaboost_with_transformer_report_df.loc[
    adaboost_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False).mean()

0.4254285714285713

##### Trying to ehence the model's performance via GridSearch

##### Decision tree has been chosen for the base estimator 

In [25]:
dt_depth_2 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=1, class_weight = "balanced")
dt_depth_9 = DecisionTreeClassifier(max_depth=9, min_samples_leaf=1, class_weight = "balanced")

In [26]:
parameters = {
    'clf__estimator__base_estimator': [dt_depth_2,dt_depth_9],
    'clf__estimator__n_estimators': [50,100]    
}

In [27]:
adaboost_transf_optimized = GridSearchCV(adaboost_with_transformer, param_grid = parameters)

In [28]:
#Attention! This cell may take a bit time to perform (around 15 minutes on typical PC)
adaboost_transf_optimized.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total= 2.5min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 1.3min
[Pipeline] .............. (step 1 of 3) Processing vect, total= 2.5min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 1.4min
[Pipeline] .............. (step 1 of 3) Processing vect, total= 2.5min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 1.3min
[Pipeline] .............. (step 1 of 3) Processing vect, total= 2.5min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 1.3min
[Pipeline] .............. (step 1 of 3) Processing vect, total= 2.5min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipel

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002885DCEEE18>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced'))))],
                                verbose=True),
             param_grid={'clf__estimator__base_estimator': [DecisionTreeClassifier(class_weight='balanced',
                                                                                   max_depth=2),
                                                            DecisionTreeClassifier(class_weight='balanced',
                                                                                   max_depth=9)],
                         'clf__estimator__n_estimators': [50, 100]})

In [29]:
y_pred_opt = adaboost_transf_optimized.predict(X_test)
y_pred_opt = pd.DataFrame(y_pred_opt, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred_opt.iloc[:,i]))

related
              precision    recall  f1-score   support

         0.0       0.53      0.46      0.49      1895
         1.0       0.83      0.87      0.85      5914

    accuracy                           0.77      7809
   macro avg       0.68      0.66      0.67      7809
weighted avg       0.76      0.77      0.76      7809

request
              precision    recall  f1-score   support

         0.0       0.90      0.93      0.92      6478
         1.0       0.59      0.52      0.56      1331

    accuracy                           0.86      7809
   macro avg       0.75      0.72      0.74      7809
weighted avg       0.85      0.86      0.85      7809

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      7771
         1.0       0.00      0.00      0.00        38

    accuracy                           0.99      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.99      0.99      0.99      780

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97      7379
         1.0       0.35      0.10      0.16       430

    accuracy                           0.94      7809
   macro avg       0.65      0.55      0.56      7809
weighted avg       0.92      0.94      0.92      7809

direct_report
              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88      6256
         1.0       0.50      0.45      0.47      1553

    accuracy                           0.80      7809
   macro avg       0.68      0.67      0.68      7809
weighted avg       0.79      0.80      0.80      7809



In [30]:
classification_reports = {}

In [31]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]));

In [32]:
adaboost_optimized_report_df = df_from_sklearn_cl_reports(classification_reports)

In [37]:
adaboost_optimized_report_df.loc[
    adaboost_optimized_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False).mean()

0.3465714285714285

In [35]:
adaboost_optimized_report_df.loc[
    adaboost_optimized_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False).mean()

0.4254285714285713

>Undersampling still gives poor results - escpecially for recall which is very important in disaster response system. Undersampling may not necesserilly work good in multioutput problems, because of possible noise beeing added to the data set.
Moreover, if we really want to balance all of the classes via undersampling we will end up with very small data set (some imbalanced classes are much less numerous than the better balanced ones). Hence the next iterration will be conducted with **´class weights´** adjustments and without any reduction of the data set size.

In [41]:
adaboost_optimized_report_df.to_csv("Adaboost_DT_GridSearch_depth2and9_estimators_50and100.csv", 
                                    index = False)